In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import csv
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from numpy.random import seed
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, GRU, Reshape
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('../input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv').sample(frac = 0.3, 
                                                                                                 random_state = 1)

# EDA and data preparation 

In [ ]:
df.head()

In [ ]:
df['text'][334]

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Missing data')
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
plt.show()

In [ ]:
df['topic'].value_counts()

In [ ]:
df.drop(df[df['topic'] == 'Библиотека'].index, inplace = True)

In [ ]:
df['tags'].value_counts()

In [ ]:
plt.figure(figsize = (12, 8))
df['title'].apply(len).hist(bins = 40)
plt.title('Length of title distribution')
plt.show()

In [ ]:
df['text'].apply(type).value_counts()

In [ ]:
df[df['text'].apply(type) == float]

In [ ]:
df.drop(df[df['text'].apply(type) == float].index, inplace = True)
df['text'].apply(type).value_counts()

In [ ]:
df['len'] = df['text'].apply(len)

In [ ]:
plt.figure(figsize = (12, 8))
df['len'].hist(bins = 130)
plt.title('Length of text distribution')
plt.show()

In [ ]:
df[df['len'] > 20000]

In [ ]:
plt.figure(figsize = (12, 8))
df['len'][df['len'] < 6000].hist(bins = 130)
plt.title('Length of text distribution')
plt.show()

In [ ]:
df[df['len'] < 300].head()

In [ ]:
df.drop(df[df['len'] < 300].index, inplace = True)

In [ ]:
df['text'].head()

In [ ]:
to_drop = {'\x03', '\x04', '\x08',  '\r', '\x0f', '\x13', '\x1d', '\x1e', '\x1f', '<', '=', '>', '@',  '^',  '`', '{', '|', '}', 
 '~', '\x7f', '\x97', '\x98', '\xa0', '¡', '©', '«', '\xad', '®', '¯', '°', '±', '´', 'µ', '·', '¸', '»', '¼', '½', '¾', 
 '¿', 'Á', 'Æ', 'Ç', 'È', 'É', 'Ë', 'Í', 'Ó', 'Ô', 'Ö', '×', 'Ø', 'Ü', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 
 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ú', 'û', 'ü', 'ý', 'þ', 'ā', 'ă', 
 'ą', 'ć', 'Č', 'č', 'Đ', 'ē', 'ė', 'ę', 'ě', 'ğ', 'ī', 'İ', 'ı', 'Ł', 'ł', 'ń', 'ņ', 'ō', 'ŏ', 'ř', 'ś', 'Ş', 'ş', 'Š', 
 'š', 'Ū', 'ų', 'ŷ', 'ź', 'ż', 'ž', 'ț', 'ɔ', 'ə', 'ɢ', 'ɪ', 'ʒ', 'ʟ', 'ʼ', '˚', '́', '̆', '̈', 'Δ', 'Θ', 'Λ', 'Ξ', 'Σ', 'έ',
 'ί', 'α', 'β', 'γ', 'δ', 'ε', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ο', 'π', 'ρ', 'ς', 'σ', 'τ', 'υ', 'χ', 'ψ', 'ω', 'ό', 'ύ',
 'ώ',  'Љ',  'љ', 'ў', 'Қ', 'қ', 'ң', 'ү', 'Ұ', 'ӧ', 'א', 'ו', 'י', 'ל', 'ם', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת', '،', '؛', '؟', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ط', 'ظ', 'ع', 'ف', 'ق', 'ك', 'ل', 'م'
 , 'ن', 'ه', 'و', 'ى', 'ي', 'پ', 'چ', 'ک', 'گ', 'ی', '۴', '۵', '۷', '۹', 'ખ', 'ફ', 'બ', 'ર', 'લ', 'સ', 'ા', 'ો', '્', 'ᴥ', 
 'ᵉ', 'ᵍ', 'ᵏ', 'ᵒ', 'ᵘ', 'ᶅ', 'ᶘ', 'ᶜ', 'ᶠ', 'Ḥ', 'ḫ', 'ả', 'ầ', 'ệ', 'ố', 'ộ', '\u2002', '\u2003', '\u2009', '\u200a', 
 '\u200b', '\u200c', '\u200d', '\u200e', '\u200f', '‘', '’', '“', '”', '„', '•', '…', '\u2028', '\u202a', '\u202c', 
 '\u202d', '\u202f', '‼', '\u2066', 'ⁱ', '⁽', '⁾', 'ⁿ', '€', '₽', '№', '™', '↓', '−', '∙', '∞', '∩', '≠', '⏳', '①', 'Ⓑ',
 'Ⓔ', 'Ⓕ', 'Ⓝ', 'Ⓞ', 'Ⓡ', 'Ⓢ', 'Ⓣ', '─', '►', '☀', '☁', '☎', '☑', '☝', '☠', '☺', '♀', '♂', '♥', '♪', '⚔', '⚖', 
 '⚡', '⚪', '⚽', '✅', '✈', '✌', '✔', '✨', '✵', '❄', '❗', '❤', '➖', '➡', '➰', '⠀', '⬅', '⬆', '⭐', '⭕', '、',
 '。', '々', '「', '」', '『', '』', '〜', '〰', 'あ', 'い', 'う', 'え', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'こ', 
 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'そ', 'た', 'だ', 'ち', 'っ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ね', 'の', 
 'は', 'ふ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'ア', 
 'イ', 'ウ', 'ェ', 'エ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'ケ', 'コ', 'ゴ', 'サ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ソ', 'ダ', 
 'チ', 'ッ', 'ツ', 'ト', 'ド', 'ナ', 'ニ', 'ネ', 'ハ', 'バ', 'パ', 'ピ', 'ブ', 'プ', 'ヘ', 'ペ', 'ボ', 'ポ', 'マ', 'ム', 'メ',
 'ャ', 'ュ', 'ョ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ン', 'ヴ', '・', 'ー', '一', '三', '上', '下', '乗', '事', '五', '人', '今',
 '从', '仕', '会', '何', '使', '俞', '光', '入', '内', '写', '出', '分', '到', '加', '動', '北', '十', '口', '可', '国', '園', 
 '土', '基', '外', '天', '女', '妙', '嬉', '子', '寝', '屋', '山', '市', '平', '幼', '度', '当', '後', '微', '思', '性', '悦', 
 '悲', '愛', '感', '成', '担', '持', '敏', '方', '日', '時', '普', '暮', '月', '有', '本', '李', '東', '极', '枚', '柒', '柯', 
 '植', '極', '様', '歧', '段', '気', '沙', '治', '洪', '活', '満', '然', '照', '片', '物', '犬', '狭', '猫', '理', '生', '産',
 '用', '番', '痢', '的', '皮', '目', '直', '真', '社', '祭', '立', '素', '組', '編', '練', '置', '考', '肌', '自', '良', '花',
 '茶', '落', '虫', '虾', '見', '视', '調', '負', '買', '賀', '購', '超', '躍', '还', '通', '遊', '達', '部', '集', '雨', '順', 
 '顔', '魚', '걸', '계', '근', '글', '기', '다', '달', '대', '동', '라', '로', '르', '리', '림', '만', '메', '모', '목', '무', 
 '물', '복', '브', '빌', '쁘', '사', '스', '실', '어', '여', '예', '오', '올', '음', '이', '접', '줄', '지', '진', '창', '코', 
 '크', '투', '티', '팅', '평', '픽', '한', '행', '헌', '회', 'ﬁ', '️', '\ufeff',  '🆘', '🌅', '🌈', '🌊', '🌌', '🌎', '🌞', 
 '🌟', '🌠', '🌱', '🌴', '🌷', '🌸', '🌹', '🌻', '🍁', '🍂', '🍃', '🍄', '🍉', '🍋', '🍌', '🍏', '🍐', '🍑', '🍒', 
 '🍓', '🍕', '🍡', '🍥', '🍦', '🍩', '🍬', '🍴', '🍷', '🍹', '🍻', '🍼', '🎀', '🎁', '🎄', '🎅', '🎆', '🎇', '🎉', '🎊', 
 '🎙', '🎥', '🎭', '🎶', '🏃', '🏋', '🏖', '🏝', '🏡', '🏺', '🏻', '🏼', '🏽', '🏾', '🏿', '🐅', '🐈', '🐍', '🐎', '🐓',
 '🐠', '🐳', '🐴', '🐶', '🐸', '🐺', '🐻', '🐾', '👀', '👇', '👉', '👊', '👋', '👌', '👍', '👏', '👑', '👒', '👖', 
 '👗', '👙', '👛', '👜', '👟', '👠', '👣', '👨', '👯', '👰', '👸', '👹', '👽', '👿', '💃', '💅', '💇', '💋', '💌', 
 '💍', '💎', '💐', '💓', '💔', '💕', '💖', '💗', '💘', '💙', '💚', '💛', '💜', '💞', '💥', '💦', '💩', '💪', '💫',
 '💭', '💯', '💸', '📍', '📝', '📲', '📷', '📸', '📺', '📻', '🔎', '🔙', '🔥', '🔪', '🔴', '🔸', '🔹', '🕊', '🕺', '🖕',
 '🖤', '🗣', '🗼', '😀', '😁', '😂', '😃', '😄', '😅', '😆', '😇', '😈', '😉', '😊', '😋', '😌', '😍', '😎', '😏', 
 '😐', '😒', '😘', '😜', '😝', '😡', '😢', '😥', '😦', '😨', '😩', '😬', '😭', '😯', '😱', '😳', '😴', '😶', '😸', 
 '😹', '😻', '😼', '😽', '🙀', '🙃', '🙄', '🙅', '🙇', '🙈', '🙋', '🙌', '🙏', '🚀', '🚂', '🚑', '🚗', '🚢', '🚨',
 '🛎', '🛒', '🤔', '🤖', '🤘', '🤣', '🤤', '🤦', '🤩', '🤪', '🤫', '🤭', '🤵', '🤷', '🤼', '🥇', '🥈', '🥉', '🥊', 
 '🥰', '🥳', '🦀', '🦄', '🦅', '🦋', '🧛', '🧜'}

In [ ]:
to_convert = {
    'Ｈ':'H',
    'Ｋ':'K',
    'Ｎ':'N',
    '𝐀':'A',
    '𝐃':'D',
    '𝐊':'K',
    '𝐍':'N',
    '𝐎':'O',
    '𝐓':'T',
    '𝐕':'V',
    '𝒆':'e',
    '𝒊':'i',
    '𝒍':'l',
    '𝒎':'m',
    '𝒐':'o',
    '𝒔':'s',
    '𝒕':'t',
    '𝒗':'v',
    '𝒚':'y',
    '𝒛':'z',
    '🇦':'A',
    '🇧':'B',
    '🇨':'C',
    '🇩':'D',
    '🇪':'E',
    '🇬':'G',
    '🇮':'I',
    '🇰':'K',
    '🇲':'M',
    '🇷':'R',
    'S':'U'
}

In [ ]:
def to_clean(text):
    for char in to_drop:
        text = text.replace(char, ' ')
    text = re.sub(r'\s+', ' ', text)
    
    for to_be_replaced, to_replace in to_convert.items():
        text = text.replace(to_be_replaced, to_replace)
    return text

In [ ]:
to_clean('𝐓𝐓𝐓𝐓𝐓𝐓𝐓75-летняя Аника 🐺🐺🐺🐺🐺🐺🐺🐺Д. из румынской деревни ')

In [ ]:
df['text'] = df['text'].apply(to_clean)

# Text to batches convertion

In [ ]:
vocab = set()

for news in df['text']:
    vocab.update(set(news))

In [ ]:
vocab = sorted(vocab)
print(vocab)
len(vocab)

In [ ]:
ind_to_char = np.array(vocab)
char_to_ind = {u:i for i, u in enumerate(vocab)}
char_to_ind

In [ ]:
encoding = lambda text: np.array([char_to_ind[c] for c in text])
df['encoded text'] = df['text'].apply(encoding)

In [ ]:
def generate_batches(text, seq_len, step):
    if len(text) < seq_len:
        return []
    else:
        sequences = []
        for i in range(0, len(text), step):
            batch = text[i:i+seq_len+1]
            if len(batch) > seq_len:
                sequences.append(text[i:i+seq_len+1])
        return sequences

In [ ]:
test ='''Как сообщило "Эхо Москвы", Конгресс русских общин (КРО) намерен идти на парламентские выборы в едином блоке
с движением Юрия Болдырева. Об этом заявил сегодня на ХI съезде КРО его лидер Дмитрий Рогозин. Он подверг резкой
критике КПРФ, заявив, что "вместо создания широкого патриотического фронта всем просто предлагают вступить в эту 
партию". Движение "Отечество", по словам Рогозина, представляет собой партию "ельцинистов без Ельцина и повторяет 
путь, уже пройденный НДР". В качестве приоритетов КРО Дмитрий Рогозин назвал "борьбу с террором, коррупцией и 
сепаратизмом."  Кроме того, делегаты проголосовали за юридическое оформление выхода КРО из состава коллективных 
членов движения "Отечество".  Комментируя это решение, лидер КРО Дмитрий Рогозин заявил, что в настоящее время из 
"Отеч'''

In [ ]:
len(test)

In [ ]:
seq_len = 120
generate_batches(test, seq_len = seq_len, step = 100)

In [ ]:
for batch in generate_batches(test, seq_len = seq_len, step = 100):
    print(len(batch))

In [ ]:
sequences = []
for text in df['encoded text']:
    batches = generate_batches(text, seq_len = seq_len, step = 400)
    sequences.extend(batches)

In [ ]:
print(f'Number of sequences: {len(sequences)}')
print(f'Example of sequence: {sequences[0]}')
print(f'Lenght of last sequence: {len(sequences[-1])}')  

In [ ]:
train_sequences, test_sequences = train_test_split(
    sequences,
    test_size=0.3, 
    random_state=42
)

In [ ]:
train_sequences_array = []
for seq in train_sequences:
    train_sequences_array.extend(seq)
    
test_sequences_array = []
for seq in test_sequences:
    test_sequences_array.extend(seq)
    
print('for training len:', len(train_sequences_array))
print('for test len:', len(test_sequences_array))

In [ ]:
train_sequences_array = np.array(train_sequences_array)
test_sequences_array = np.array(test_sequences_array)
print(train_sequences_array)

train_char_dataset = tf.data.Dataset.from_tensor_slices(train_sequences_array)
train_sequences = train_char_dataset.batch(seq_len+1, drop_remainder=True) 

test_char_dataset = tf.data.Dataset.from_tensor_slices(test_sequences_array)
test_sequences = test_char_dataset.batch(seq_len+1, drop_remainder=True) 

In [ ]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [ ]:
train_dataset = train_sequences.map(create_seq_targets)
test_dataset = test_sequences.map(create_seq_targets)

In [ ]:
batch_size = 128
buffer_size = 10000

train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
test_dataset = test_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [ ]:
vocab_size = len(vocab)
vocab_size

# Modeling

In [ ]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
def create_model(vocab_size, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 64, batch_input_shape=[batch_size, None]))
    model.add(LSTM(512,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(LSTM(512,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(LSTM(512,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [ ]:
def generate_text(model, start_seed,num_generate=100,temperature=1.0):

    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [ ]:
seed(101)
set_seed(101)

es = EarlyStopping(monitor='val_loss',patience=4)

model = create_model(
    vocab_size = vocab_size,
    batch_size=batch_size
)

model.compile(optimizer='adam', loss=sparse_cat_loss) 

model.summary()

### 3 epochs

In [ ]:
model.fit(
    train_dataset,
    epochs = 3,
    validation_data = test_dataset,
    callbacks=[es] 
)

model.save('LSTM 3x512 3e.h5')

histo = pd.DataFrame(model.history.history)
metric = ['loss', 'val_loss']
histo[metric].plot()
plt.title(' and '.join(metric))
plt.show()

In [ ]:
model_pred = create_model(vocab_size, batch_size=1)
model_pred.load_weights('LSTM 3x512 3e.h5')
model_pred.build(tf.TensorShape([1, None]))

In [ ]:
string = "А"
size = 10000
temp = 1
print(generate_text(model_pred,string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.5
print(generate_text(model_pred,string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.2
print(generate_text(model_pred,string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.1
print(generate_text(model_pred,string, size, temp))

### 10 epochs

In [ ]:
model.fit(
    train_dataset,
    epochs = 7,
    validation_data = test_dataset,
    callbacks=[es] 
)

model.save('LSTM 3x512 10e.h5')

histo = pd.DataFrame(model.history.history)
metric = ['loss', 'val_loss']
histo[metric].plot()
plt.title(' and '.join(metric))
plt.show()

In [ ]:
model_pred10 = create_model(vocab_size, batch_size=1)
model_pred10.load_weights('LSTM 3x512 10e.h5')
model_pred10.build(tf.TensorShape([1, None]))

In [ ]:
string = "А"
size = 10000
temp = 1
print(generate_text(model_pred10, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.5
print(generate_text(model_pred10, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.2
print(generate_text(model_pred10, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.1
print(generate_text(model_pred10, string, size, temp))

### 20 epochs

In [ ]:
model.fit(
    train_dataset,
    epochs = 10,
    validation_data = test_dataset,
    callbacks=[es] 
)

model.save('LSTM 3x512 20e.h5')

histo = pd.DataFrame(model.history.history)
metric = ['loss', 'val_loss']
histo[metric].plot()
plt.title(' and '.join(metric))
plt.show()

In [ ]:
model_pred20 = create_model(vocab_size, batch_size=1)
model_pred20.load_weights('LSTM 3x512 20e.h5')
model_pred20.build(tf.TensorShape([1, None]))

In [ ]:
string = "А"
size = 10000
temp = 1
print(generate_text(model_pred20, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.5
print(generate_text(model_pred20, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.2
print(generate_text(model_pred20, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.1
print(generate_text(model_pred20, string, size, temp))

### 30 epochs

In [ ]:
model.fit(
    train_dataset,
    epochs = 10,
    validation_data = test_dataset,
    callbacks=[es] 
)

model.save('LSTM 3x512 30e.h5')

histo = pd.DataFrame(model.history.history)
metric = ['loss', 'val_loss']
histo[metric].plot()
plt.title(' and '.join(metric))
plt.show()

In [ ]:
model_pred30 = create_model(vocab_size, batch_size=1)
model_pred30.load_weights('LSTM 3x512 30e.h5')
model_pred30.build(tf.TensorShape([1, None]))

In [ ]:
string = "А"
size = 10000
temp = 1
print(generate_text(model_pred30, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.5
print(generate_text(model_pred30, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.2
print(generate_text(model_pred30, string, size, temp))

In [ ]:
string = "А"
size = 10000
temp = 0.1
print(generate_text(model_pred30, string, size, temp))